In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: AWSSDK.BedrockRuntime, 3.7.302.7"
#r "nuget: Rockhead.Extensions, 0.1.5-preview"

# Support classes, Semantic Kernel integration etc.
## Note on Text Completions API vs Messages API
- seems Text Completions API is the "older" one having the "low level" text as input
    - e.g. `"prompt": "Human:\n\n What is quantum mechanics? \n\n Assistant:\n\n",`
- while Messages API is more akin to what's known from the other Quokka project
    - e.g. `"messages": [{"role": "user","content": [{"type": "text","text": "What is quantum mechanics?"}]}],`

In [ ]:
using System;
using System.IO;
using System.Threading;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Text.Json.Serialization;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Amazon;
using Amazon.Util;
using Amazon.BedrockRuntime;
using Amazon.BedrockRuntime.Model;
using Rockhead.Extensions; // TODO: consider contributing
using Rockhead.Extensions.Meta;
using Rockhead.Extensions.Anthropic;

public class ClaudeTextGenerationResponseContent
{
	[JsonPropertyName("type")]
	public string Type { get; set; }

	[JsonPropertyName("text")]
	public string Text { get; set; }
}

public class ClaudeTextGenerationResponseUsage
{
	[JsonPropertyName("input_tokens")]
	public int InputTokens { get; set; }

	[JsonPropertyName("output_tokens")]
	public int OutputTokens { get; set; }
}

public class ClaudeTextGenerationResponse
{
	[JsonPropertyName("id")]
	public string Id { get; set; }

	[JsonPropertyName("type")]
	public string Type { get; set; }

	[JsonPropertyName("role")]
	public string Role { get; set; }

	[JsonPropertyName("content")]
	public List<ClaudeTextGenerationResponseContent> Content { get; set; }

	[JsonPropertyName("model")]
	public string Model { get; set; }

	[JsonPropertyName("stop_reason")]
	public string StopReason { get; set; }

	[JsonPropertyName("stop_sequence")]
	public string StopSequence { get; set; }

	[JsonPropertyName("usage")]
	public ClaudeTextGenerationResponseUsage Usage { get; set; }

	public string Completion => this.Content[0]?.Text ?? string.Empty;
	public string Stop => this.StopSequence ?? this.StopReason;
}

public class SemanticBedrock : Microsoft.SemanticKernel.TextGeneration.ITextGenerationService
{
	public IReadOnlyDictionary<string, object?> Attributes => new Dictionary<string, object?>();

	private string ModelId { get; set; }
	private string AwsAccessKeyId { get; set; }
	private string AwsSecretAccessKey { get; set; }

	public SemanticBedrock(string modelId, string awsAccessKeyId, string awsSecretAccessKey)
	{
		Console.WriteLine($"SemanticBedrock constructor: modelId: {modelId}, awsAccessKeyId: {!string.IsNullOrWhiteSpace(awsAccessKeyId)}, awsSecretAccessKey: {!string.IsNullOrWhiteSpace(awsSecretAccessKey)}");
		this.ModelId = modelId;
		this.AwsAccessKeyId = awsAccessKeyId;
		this.AwsSecretAccessKey = awsSecretAccessKey;
	} // TODO ctor: https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/Connectors/Connectors.Google/Services/GoogleAIGeminiChatCompletionService.cs#L31

	public IAsyncEnumerable<StreamingTextContent> GetStreamingTextContentsAsync(string prompt, PromptExecutionSettings? executionSettings = null, Kernel? kernel = null, CancellationToken cancellationToken = default)
	{
		Console.WriteLine("GetStreamingTextContentsAsync");
		throw new NotImplementedException();
	}

	public Task<IReadOnlyList<TextContent>> GetTextContentsAsync(string prompt, PromptExecutionSettings? executionSettings = null, Kernel? kernel = null, CancellationToken cancellationToken = default)
	{
		Console.WriteLine($"GetTextContentsAsync, prompt: {prompt}");

		if (string.IsNullOrWhiteSpace(this.AwsAccessKeyId) || string.IsNullOrWhiteSpace(this.AwsSecretAccessKey))
		{
			Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, exiting...");
			throw new Exception("AWSAccessKeyId or AWSSecretAccessKey is not set");
		}
		AmazonBedrockRuntimeClient awsClient = new(this.AwsAccessKeyId, this.AwsSecretAccessKey, RegionEndpoint.USEast1);
		string payload = new JsonObject()
		{
			{ "messages",
				new JsonArray
				{
					new JsonObject
					{
						{ "role", "user" },
						{ "content",
							new JsonArray
							{
								new JsonObject
								{
									{ "type", "text" },
									{ "text", prompt }
								}
							}
						}
					}
				}
			},
			{ "max_tokens", 512 },
			{ "temperature", 0.5 },
			{ "anthropic_version", "bedrock-2023-05-31" }
		}.ToJsonString();

		try
		{
			var response = awsClient.InvokeModelAsync(new InvokeModelRequest
			{
				ModelId = this.ModelId,
				Body = AWSSDKUtils.GenerateMemoryStreamFromString(payload),
				ContentType = "application/json",
				Accept = "application/json"
			}).GetAwaiter().GetResult();
			Console.WriteLine(response);
			Console.WriteLine(response?.HttpStatusCode);
			Console.WriteLine(response?.Body?.ToString());

			// use system.text.json to parse the response into testResponse object
			var jsonString = (new StreamReader(response.Body).ReadToEndAsync()).GetAwaiter().GetResult();
			Console.WriteLine($"Response jsonString: {jsonString}");

			// {"id":"msg_011NiLrziFXmBy9TJxMketRD","type":"message","role":"assistant","content":[{"type":"text","text":"AI (Artificial Intelligence) is awesome because ... (removed for brevity)"}],"model":"claude-3-sonnet-28k-20240229","stop_reason":"end_turn","stop_sequence":null,"usage":{"input_tokens":15,"output_tokens":141}}
			var testResponse = JsonSerializer.Deserialize<ClaudeTextGenerationResponse>(jsonString) ?? new ClaudeTextGenerationResponse();
			Console.WriteLine($"Deserialized response: {testResponse}");
			Console.WriteLine($"Id: {testResponse.Id}");
			Console.WriteLine($"Type: {testResponse.Type}");
			Console.WriteLine($"Role: {testResponse.Role}");
			foreach (var content in testResponse.Content)
			{
				Console.WriteLine($"Content text: {content.Text}");
				Console.WriteLine($"Content type: {content.Type}");
			}
			Console.WriteLine($"Completion: {testResponse.Completion}");
			Console.WriteLine($"Model: {testResponse.Model}");
			Console.WriteLine($"Usage: {testResponse.Usage}");
			Console.WriteLine($"Stop: {testResponse.Stop}");
			Console.WriteLine($"StopSequence: {testResponse.StopSequence}");
			Console.WriteLine($"StopReason: {testResponse.StopReason}");

			return Task.FromResult<IReadOnlyList<TextContent>>(new List<TextContent> { new(testResponse.Completion) });
		}
		catch (Exception ex)
		{
			Console.WriteLine($"Error invoking model: {ex.Message}");
			return Task.FromResult<IReadOnlyList<TextContent>>(new List<TextContent> { new("Error invoking model") });
		}
	}
}

# Load API keys from dotnet User-Secrets and setup Semantic Kernel

In [ ]:
using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.TextGeneration;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;


var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile(Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
        "Microsoft",
        "UserSecrets",
        "2b6c897c-02c0-4c8e-91a8-cb61113a6acc", // comes from the project file QuokkaVec.Console.csproj
        "secrets.json")
    );

IConfigurationRoot configuration = builder.Build();
string awsAccessKeyId = configuration["AWSAccessKeyId"] ?? string.Empty;
string awsSecretAccessKey = configuration["AWSSecretAccessKey"] ?? string.Empty;

if (string.IsNullOrWhiteSpace(awsAccessKeyId) || string.IsNullOrWhiteSpace(awsSecretAccessKey))
{
	Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, use 'dotnet user-secrets init' and 'dotnet user-secrets set <key> <value>' to set the key, exiting...");
	return;
}

Console.WriteLine("API Keys set");

#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var kernelBuilder = Kernel.CreateBuilder();
// kernelBuilder.Services.AddSingleton<ITextGenerationService, SemanticBedrock>();
kernelBuilder.Services.AddSingleton<ITextGenerationService, SemanticBedrock>((_) => new SemanticBedrock("anthropic.claude-3-sonnet-20240229-v1:0", awsAccessKeyId, awsSecretAccessKey));//"anthropic.claude-3-sonnet-20240229-v1:0";//"meta.llama3-70b-instruct-v1:0";
// kerbelBuilder.AddOpenAITextEmbeddingGeneration("text-embedding-3-small", openAIAPIKey);
var kernel = kernelBuilder.Build();

// var embeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

const string functionDefinition = "Write one paragraph on {{$input}}";
var paragraphWritingFunction = kernel.CreateFunctionFromPrompt(functionDefinition);

const string input = "Why AI is awesome";
Console.WriteLine($"Function input: {input}\n");
var result = await paragraphWritingFunction.InvokeAsync(kernel, new() { ["input"] = input });
Console.WriteLine($"Function output: {result}");
// kernel.InvokePromptAsync("Write one paragraph on Why AI is awesome", cancellationToken: default).Wait();

# Bedrock Claude Chat service



In [ ]:
using System;
using System.IO;
using System.Threading;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Text.Json.Serialization;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Amazon;
using Amazon.Util;
using Amazon.BedrockRuntime;
using Amazon.BedrockRuntime.Model;
using Rockhead.Extensions; // TODO: consider contributing
using Rockhead.Extensions.Meta;
using Rockhead.Extensions.Anthropic;

public class SemanticBedrockChatCompletionService : Microsoft.SemanticKernel.ChatCompletion.IChatCompletionService
{
	private string ModelId { get; set; }
	private string AwsAccessKeyId { get; set; }
	private string AwsSecretAccessKey { get; set; }

	public IReadOnlyDictionary<string, object?> Attributes => new Dictionary<string, object?>();

	public SemanticBedrockChatCompletionService(string modelId, string awsAccessKeyId, string awsSecretAccessKey)
	{
		Console.WriteLine($"SemanticBedrockChatCompletionService constructor: modelId: {modelId}, awsAccessKeyId: {!string.IsNullOrWhiteSpace(awsAccessKeyId)}, awsSecretAccessKey: {!string.IsNullOrWhiteSpace(awsSecretAccessKey)}");
		this.ModelId = modelId;
		this.AwsAccessKeyId = awsAccessKeyId;
		this.AwsSecretAccessKey = awsSecretAccessKey;
	}

	public Task<IReadOnlyList<ChatMessageContent>> GetChatMessageContentsAsync(ChatHistory chatHistory, PromptExecutionSettings? executionSettings = null, Kernel? kernel = null, CancellationToken cancellationToken = default)
	{
		Console.WriteLine("GetChatMessageContentsAsync");

		if (string.IsNullOrWhiteSpace(this.AwsAccessKeyId) || string.IsNullOrWhiteSpace(this.AwsSecretAccessKey))
		{
			Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, exiting...");
			throw new Exception("AWSAccessKeyId or AWSSecretAccessKey is not set");
		}
		AmazonBedrockRuntimeClient awsClient = new(this.AwsAccessKeyId, this.AwsSecretAccessKey, RegionEndpoint.USEast1);

		// turn chatHistory parameter into the json payload
		JsonArray messages = new();
		foreach (var chatMessage in chatHistory)
		{
			JsonArray contentArray = new();
			//contentArray.Add(chatMessage.Items.OfType<TextContent>().Select(item =>
			//				new JsonObject
			//				{
			//					{ "type", "text" },
			//					{ "text", item.Text }
			//				})); // doesn't work for some reason
			contentArray.Add(new JsonObject
			{
				{ "type", "text" },
				{ "text", chatMessage.Content } // simplified for now down to just the content
			});

			messages.Add(
				new JsonObject
				{
					{ "role", chatMessage.Role.Label },
					{ "content", contentArray }
				});
			Console.WriteLine($"   - Added ChatMessage: {chatMessage.Role.Label} - {chatMessage.Content}");
		}

		// prepare function/tool/plugin call data
		// see https://docs.anthropic.com/en/docs/tool-use
		// looks like function calling is not available on Claude on Bedrock yet
		// https://repost.aws/questions/QUHOlMnUGjTQ28g9938DuyVg/bedrock-function-calling-feature
		// workaround? https://medium.com/@daniellefranca96/running-a-langchain-agent-on-bedrock-claude-using-the-model-function-calling-5f400a8f0d62
		// or possibly what https://github.com/Cysharp/Claudia?tab=readme-ov-file#aws-bedrock is doing
		// see https://docs.anthropic.com/en/docs/legacy-tool-use
		// for fun below just let's add some test system messages
		var tools = executionSettings?.ExtensionData?["Plugins"] as List<KernelPlugin> ?? new();

		if (tools.Any())
		{
			Console.WriteLine($"Tools found: {tools.Count}");

			// hack:
			messages.Add(
			new JsonObject
			{
					{ "role", "assistant" },
					{ "content", new JsonArray { new JsonObject { { "type", "text" }, { "text", "Can you provide any tools I can use?" } } } }
			});

			string functionsDescription = tools.Select(tool =>
			$"""
			<tool_description>
			<tool_name>{tool.Name}</tool_name>
			<description>
			{tool.Description}
			</description>
			<parameters>
			</parameters>
			</tool_description>

			"""
			).Aggregate((a, b) => a + b);
			// TODO: {tool.FirstOrDefault()?.Metadata?.Parameters}

			messages.Add(
				new JsonObject
				{
					{ "role", "user" }, // Claude doesn't support tool role seems?
					{ "content", new JsonArray { new JsonObject { { "type", "text" }, { "text", functionsDescription } } } }
				});
		}
		else
		{
			Console.WriteLine("No tools found");
		}


		string payload = new JsonObject()
		{
			{ "messages", messages },
			{ "max_tokens", 2048 },
			{ "temperature", 0.5 },
			{ "anthropic_version", "bedrock-2023-05-31" }
		}.ToJsonString();

		try
		{
			// TODO: consider using RockheadExtension methods instead of direct call
			//awsClient.InvokeClaudeMessagesAsync(Model.Claude.Claude3Sonnet, new ClaudeMessage { Content = null, Role = "user"}, new ClaudeMessagesConfig() { })
			var response = awsClient.InvokeModelAsync(new InvokeModelRequest
			{
				ModelId = this.ModelId,
				Body = AWSSDKUtils.GenerateMemoryStreamFromString(payload),
				ContentType = "application/json",
				Accept = "application/json"
			}).GetAwaiter().GetResult();
			Console.WriteLine(response);
			Console.WriteLine(response?.HttpStatusCode);
			Console.WriteLine(response?.Body?.ToString());

			// use system.text.json to parse the response into testResponse object
			var jsonString = (new StreamReader(response.Body).ReadToEndAsync()).GetAwaiter().GetResult();
			Console.WriteLine($"  - Response jsonString: {jsonString}");

			// {"id":"msg_011NiLrziFXmBy9TJxMketRD","type":"message","role":"assistant","content":[{"type":"text","text":"AI (Artificial Intelligence) is awesome because ... (removed for brevity)"}],"model":"claude-3-sonnet-28k-20240229","stop_reason":"end_turn","stop_sequence":null,"usage":{"input_tokens":15,"output_tokens":141}}
			var claudeResponse = JsonSerializer.Deserialize<ClaudeTextGenerationResponse>(jsonString) ?? new ClaudeTextGenerationResponse();
			Console.WriteLine($"  - Deserialized response: {claudeResponse}");
			Console.WriteLine($"  - Id: {claudeResponse.Id}");
			Console.WriteLine($"  - Type: {claudeResponse.Type}");
			Console.WriteLine($"  - Role: {claudeResponse.Role}");
			foreach (var content in claudeResponse.Content)
			{
				Console.WriteLine($"   - Content text: {content.Text}");
				Console.WriteLine($"   - Content type: {content.Type}");
			}
			Console.WriteLine($"  - Completion: {claudeResponse.Completion}");
			Console.WriteLine($"  - Model: {claudeResponse.Model}");
			Console.WriteLine($"  - Usage: {claudeResponse.Usage}");
			Console.WriteLine($"  - Stop: {claudeResponse.Stop}");
			Console.WriteLine($"  - StopSequence: {claudeResponse.StopSequence}");
			Console.WriteLine($"  - StopReason: {claudeResponse.StopReason}");

			AuthorRole ClaudeAuthorStringToKernelRole(string authorString)
			{
				switch (authorString)
				{
					case "assistant":
						return AuthorRole.Assistant;
					case "user":
						return AuthorRole.User;
					case "system":
						return AuthorRole.System;
					case "tool":
						return AuthorRole.Tool;
					default:
						return AuthorRole.Assistant;
				}
			}

			return Task.FromResult<IReadOnlyList<ChatMessageContent>>(
				claudeResponse.Content.Select(content =>
					new ChatMessageContent(
						ClaudeAuthorStringToKernelRole(claudeResponse.Role),
						content.Text,
						modelId: claudeResponse.Model)
					)
				.ToList());
		}
		catch (Exception ex)
		{
			Console.WriteLine($"Error invoking model: {ex.Message}");
			return Task.FromResult<IReadOnlyList<ChatMessageContent>>(new List<ChatMessageContent> { new(AuthorRole.Assistant, "Error invoking model") });
		}
	}

	public IAsyncEnumerable<StreamingChatMessageContent> GetStreamingChatMessageContentsAsync(ChatHistory chatHistory, PromptExecutionSettings? executionSettings = null, Kernel? kernel = null, CancellationToken cancellationToken = default)
	{
		Console.WriteLine("GetStreamingChatMessageContentsAsync");
		throw new NotImplementedException();
	}
}

In [ ]:
using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.TextGeneration;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;


var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile(Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
        "Microsoft",
        "UserSecrets",
        "2b6c897c-02c0-4c8e-91a8-cb61113a6acc", // comes from the project file QuokkaVec.Console.csproj
        "secrets.json")
    );

IConfigurationRoot configuration = builder.Build();
string awsAccessKeyId = configuration["AWSAccessKeyId"] ?? string.Empty;
string awsSecretAccessKey = configuration["AWSSecretAccessKey"] ?? string.Empty;

if (string.IsNullOrWhiteSpace(awsAccessKeyId) || string.IsNullOrWhiteSpace(awsSecretAccessKey))
{
	Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, use 'dotnet user-secrets init' and 'dotnet user-secrets set <key> <value>' to set the key, exiting...");
	return;
}

Console.WriteLine("API Keys set");

#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.Services.AddSingleton<IChatCompletionService, SemanticBedrockChatCompletionService>((_) =>
    new("anthropic.claude-3-sonnet-20240229-v1:0", awsAccessKeyId, awsSecretAccessKey));
// kerbelBuilder.AddOpenAITextEmbeddingGeneration("text-embedding-3-small", openAIAPIKey);
var kernel = kernelBuilder.Build();
// var embeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

ChatHistory chatHistory = new();
chatHistory.AddUserMessage("Hi, can you tell me what's the date today?");

// chat
var chatService = kernel.GetRequiredService<IChatCompletionService>();

var chatResult = await chatService.GetChatMessageContentsAsync(chatHistory);
chatHistory.AddAssistantMessage(chatResult.FirstOrDefault()?.Content ?? "No response"); // just for convenience simplified to first message
foreach (var chatMessage in chatResult)
{
    Console.WriteLine($"ChatMessage: {chatMessage.Role} - {chatMessage.Content}");
}

In [ ]:
using System.ComponentModel;

public class SimpleDateTimePlugin
{
    [KernelFunction]
    [Description("Returns the current date and time.")]
    public string GetDateTime()
    {
        return DateTime.Now.ToString();
    }
}

KernelPlugin dateTimePlugin = kernel.CreatePluginFromType<SimpleDateTimePlugin>();
kernel.Plugins.Add(dateTimePlugin);

chatHistory.AddUserMessage("Can you tell me the date and time now?");
var chatResult = await chatService.GetChatMessageContentsAsync(chatHistory, executionSettings: new PromptExecutionSettings { ExtensionData = new Dictionary<string, object?> { ["Plugins"] = new List<KernelPlugin> { dateTimePlugin } } });
chatHistory.AddAssistantMessage(chatResult.FirstOrDefault()?.Content ?? "No response"); // just for convenience simplified to first message

chatHistory

# TODO: Plugin and Planner
- https://devblogs.microsoft.com/semantic-kernel/introducing-api-manifest-plugins-for-semantic-kernel-2/
- consider https://github.com/Cysharp/Claudia and https://github.com/Cysharp/Claudia?tab=readme-ov-file#aws-bedrock

In [ ]:
using System.ComponentModel;
using Microsoft.SemanticKernel;

// note, copied from https://github.com/microsoft/autogen/blob/main/dotnet/sample/AutoGen.BasicSamples/Example10_SemanticKernel.cs
public class SimpleLightPlugin
{
    public bool IsOn { get; set; } = false;

    [KernelFunction]
    [Description("Gets the state of the light.")]
    public string GetState() => this.IsOn ? "on" : "off";

    [KernelFunction]
    [Description("Changes the state of the light.'")]
    public string ChangeState(bool newState)
    {
        this.IsOn = newState;
        var state = this.GetState();

        Console.WriteLine($"[Light is now {state}]");

        return state;
    }
}


// prepare Kernel plugin
KernelPlugin lightPlugin = kernel.CreatePluginFromType<SimpleLightPlugin>();
kernel.Plugins.Add(lightPlugin);

// TODO: var skAgent = kernel.ToSemanticKernelAgent(name: "assistant", systemMessage: "You control the light", settings);

string goal =
"""
Control the light, turn it on if it's off, and leave it on if it's on.
""";

// prepare planner
// FunctionCallingPlanner

In [ ]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

// var executionSettings = new OpenAIPromptExecutionSettings  //TODO bedrock settings
// {
//     MaxTokens = 2000,
//     Temperature = 0.7,
//     TopP = 0.5
// };
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt);//, executionSettings);
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

In [ ]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);

arguments["userInput"] = "Alright, but I'm looking for something in the fantasy genre";

var answer = await chatFunction.InvokeAsync(kernel, arguments);

var result = $"\nUser: {arguments["userInput"]}\nAI: {answer}\n";
history += result;

arguments["history"] = history;

// Show the response
Console.WriteLine(result);

In [ ]:
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.10.0-alpha"
#r "nuget: System.Interactive.Async, 6.0.1"

using System.Linq;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Memory functionality is experimental
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0050
var memoryBuilder = new MemoryBuilder();

//TODO replace with Bedrock
memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", openAIAPIKey);

memoryBuilder.WithMemoryStore(new VolatileMemoryStore()); // TODO try some Bedrock memory store
var memory = memoryBuilder.Build();

const string MemoryCollectionName = "aboutMe";
await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andrea");

var response = await memory.SearchAsync(MemoryCollectionName, "What is my name?").FirstOrDefaultAsync();
Console.WriteLine(response);
Console.WriteLine(response?.Display());

In [ ]:
#r "nuget: Qdrant.Client, 1.9.0"

In [ ]:
docker start quokkavec-qdrant

# Setup Qdrant vector DB, setup Collection and fill it with embeddings

In [ ]:
var pathToIndex = "c:/temp/quokkavec-test";

In [ ]:
using Qdrant.Client;
using Qdrant.Client.Grpc;

#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var collectionName = "test";
var qdrantClient = new QdrantClient("localhost");
if (!await qdrantClient.CollectionExistsAsync(collectionName))
{
	await qdrantClient.CreateCollectionAsync(collectionName, new VectorParams { Size = 1536, Distance = Distance.Cosine });
}

// index all files in the folder
var files = Directory.GetFiles(pathToIndex, "*", SearchOption.AllDirectories);
int i = 0;
foreach (var file in files)
{
	var text = await File.ReadAllTextAsync(file);
	// take only first 1k characters
	text = text.Substring(0, Math.Min(1000, text.Length));
	ReadOnlyMemory<float> embedding = await embeddingGenerationService.GenerateEmbeddingAsync(text);

	var points = new List<PointStruct>()
	{
		new PointStruct
		{
			Id = new PointId { Num = (ulong)i },
			Vectors = embedding.ToArray(),
			Payload = { { "fileName", file } }
		}
	};
	var updateResult = await qdrantClient.UpsertAsync(collectionName, points);
	Console.WriteLine($"File {file} indexed, result: {updateResult}");

	i++;
}

In [ ]:
string searchQuery = "Latest news";

In [ ]:
#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var queryVector = await embeddingGenerationService.GenerateEmbeddingAsync(searchQuery);
var searchResult = await qdrantClient.SearchAsync(collectionName, queryVector.ToArray(), limit: 2);

foreach (var result in searchResult)
{
	Console.WriteLine($"File: {result.Payload["fileName"]}, Score: {result.Score}");
}